In [1]:
from mom import influxDB_utils as influx
from mom import Mandelbrot
from influxdb_client import Point
from datetime import datetime, timezone
import pandas as pd

In [3]:
#test influx variables
#influx.get_org()

In [43]:
#collect available historical data
c = "BTC"
MIN, HOUR, DAY, WEEK, MONTH = Mandelbrot.historical_datasets(crypto=c, curr="EUR")

downloading the EUR price/m for BTC from https://www.cryptodatadownload.com/cdd/Binance_BTCEUR_minute.csv....
downloading the EUR price/m for BTC from yfinance....
merged yf and cdd
downloading the EUR price/d for BTC from https://www.cryptodatadownload.com/cdd/Binance_BTCEUR_d.csv....
downloading the EUR price/d for BTC from yfinance....
merged yf and cdd


/home/christiane/git/MOM_Crypto_Bot/crypto_risk_analytics/src/analysis/Mandelbrot.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HOUR.loc[:,"delta_high"] = HOUR["high"].diff()
/home/christiane/git/MOM_Crypto_Bot/crypto_risk_analytics/src/analysis/Mandelbrot.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HOUR.loc[:,"delta_low"] = HOUR["low"].diff()
/home/christiane/git/MOM_Crypto_Bot/crypto_risk_analytics/src/analysis/Mandelbrot.py:92: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [5]:
MIN.head()

,unix,volume,high,low,delta,delta_log,return,return_log,currency,asset,interval
date,,,,,,,,,,,
2021-01-03 18:58:00,1609700280,16.31624,728.83,726.31,NaN,NaN,NaN,NaN,EUR,ETH,Minute
2021-01-03 18:59:00,1609700340,16.31751,729.14,727.46,0.31,-0.508638,0.000425,0.000425,EUR,ETH,Minute
2021-01-03 19:00:00,1609700400,92.63962,728.79,724.52,-0.35,0.455932,-0.000480,-0.000480,EUR,ETH,Minute
2021-01-03 19:01:00,1609700460,121.72397,726.15,722.11,-2.64,-0.421604,-0.003622,-0.003629,EUR,ETH,Minute
2021-01-03 19:02:00,1609700520,33.70983,728.00,723.00,1.85,0.267172,0.002548,0.002544,EUR,ETH,Minute


In [4]:
DF = pd.read_csv(f"historical_data/2020_m_{c}_to_EUR.csv")
DF.head()

,Unnamed: 0,Unix Timestamp,Low,High,Open,Close,Volume,Asset,Curr,Date
0,0,1577851140,116.26,116.26,116.26,116.26,0.050685,ETH,EUR,2020-01-01 03:59:00
1,1,1577851020,116.26,116.26,116.26,116.26,0.070078,ETH,EUR,2020-01-01 03:57:00
2,2,1577850900,116.34,116.34,116.34,116.34,0.436275,ETH,EUR,2020-01-01 03:55:00
3,3,1577850720,116.34,116.34,116.34,116.34,0.036178,ETH,EUR,2020-01-01 03:52:00
4,4,1577850660,116.24,116.24,116.24,116.24,1.507977,ETH,EUR,2020-01-01 03:51:00


In [2]:
#combine with csv data



def old_csv(c,g,year,interv):

        
    DF = pd.read_csv(f"historical_data/{year}_{g}_{c}_to_EUR.csv")
    DF.columns = map(str.lower, DF.columns)
    DF["currency"] = DF["curr"]
    #delta etc
    DF["delta"] = DF["high"].diff()
    DF["delta_log"] = Mandelbrot.log10(DF["delta"])
    DF["return"] = Mandelbrot.compute_returns(DF["high"])[0]
    DF["return_log"] = Mandelbrot.compute_returns(DF["high"])[1]

    #datetype M
    DF['date'] = pd.to_datetime(DF['unix timestamp'], unit='s')
    rfc3339 = []
    for i in DF["unix timestamp"]:
        dt = datetime.fromtimestamp(i, tz= timezone.utc)
        date = dt.isoformat().replace("+00:00","Z")
        rfc3339.append(date)
    DF["date_rfc"] = rfc3339

    DF = DF.set_index("date")
    DF["interval"] = interv
    co =['high','volume','asset', 'currency', 'delta', 'delta_log', 'return',
           'return_log', 'interval']
    DF = DF[co]

    # Ensure numeric columns are floats
    for col in ["high","low","volume", "delta", "delta_log", "return", "return_log"]:
        if col in DF.columns:
            DF[col] = pd.to_numeric(DF[col], errors="coerce")

    # Ensure tag columns are strings
    for col in ["asset", "currency", "interval"]:
        if col in DF.columns:
            DF[col] = DF[col].astype(str)

    
    DF.index = pd.to_datetime(DF.index, utc=True)

    return DF

#print(DF.head())
#print(DF.dtypes)

In [4]:
influx.write_dataframe(old_csv("BTC","m","2016","Minute"),measurement="Minute")

Writing to InfluxDB bucket 'CryptoPrices-Minute'
Wrote 613 rows to InfluxDB bucket 'CryptoPrices-Minute'


In [5]:
DF = old_csv("BTC","m","2016","Minute")

In [8]:
pd

613


In [39]:
from influxdb_client import Point

query_api = influx.get_client().query_api()

q = f'''
from(bucket: "{influx.INFLUX_BUCKET}")
  |> range(start: -1d)
  |> filter(fn: (r) => r._measurement == "Minute")
  |> limit(n:5)
'''
tables = query_api.query(q)
for table in tables:
    for record in table.records:
        print(record)


In [6]:
DF.index.dtype.kind

'M'

In [18]:
DF = old_csv("BTC","m","2016","Minute")
DF
influx.write_dataframe(DF, measurement = "Minute")

Writing to InfluxDB bucket 'CryptoPrices-Minute'
Wrote 613 rows to InfluxDB bucket 'CryptoPrices-Minute'


In [9]:
for year in ["2016","2017","2018"]:
#for year in ["2019","2020","2021"]:
#for year in ["2022","2023"]:
#for year in ["2024","2025"]:
    for g in ["m","h","d"]:
        if g == "m":
            interv = "Minute"
        elif g == "h":
            interv = "Hour"
        else:
            interv = "Day"
        
        for c in ["BTC"]:#,"SHIB"]:
            try:
                
                DF = old_csv(c,g,year,interv)
                print(DF.head())
            except Exception:
                print(year,c,g)
                pass
            
            
                #print(DF.head())
            if len(DF) > 0:
                print(year,c,len(DF))
                influx.write_dataframe(DF, measurement = interv)

                             high  volume asset currency  delta  delta_log  \
date                                                                         
2016-12-31 01:59:00+00:00  908.08    0.17   BTC      EUR    NaN        NaN   
2016-12-31 01:56:00+00:00  908.15    0.02   BTC      EUR   0.07  -1.154902   
2016-12-31 01:55:00+00:00  908.15    0.16   BTC      EUR   0.00   0.000000   
2016-12-31 01:54:00+00:00  908.19    0.18   BTC      EUR   0.04  -1.397940   
2016-12-31 01:52:00+00:00  907.00    0.02   BTC      EUR  -1.19  -0.075547   

                             return  return_log interval  
date                                                      
2016-12-31 01:59:00+00:00       NaN         NaN   Minute  
2016-12-31 01:56:00+00:00  0.000077    0.000077   Minute  
2016-12-31 01:55:00+00:00  0.000000    0.000000   Minute  
2016-12-31 01:54:00+00:00  0.000044    0.000044   Minute  
2016-12-31 01:52:00+00:00 -0.001310   -0.001311   Minute  
2016 BTC 613
Writing to InfluxDB bucket 

In [3]:
DAY.columns

Index(['unix', 'volume', 'high', 'low', 'delta', 'delta_log', 'return',
       'return_log', 'currency', 'asset', 'interval'],
      dtype='object')

In [4]:
DAY.index.dtype.kind

'M'

In [44]:
for INTERVAL in [MIN,HOUR,DAY,WEEK,MONTH]:
    influx.write_dataframe(INTERVAL[co])

Wrote 824578 rows to InfluxDB bucket 'CryptoPrices'
Wrote 23277 rows to InfluxDB bucket 'CryptoPrices'
Wrote 4004 rows to InfluxDB bucket 'CryptoPrices'
Wrote 572 rows to InfluxDB bucket 'CryptoPrices'
Wrote 132 rows to InfluxDB bucket 'CryptoPrices'


In [8]:
print(MIN[co].dtypes)
print(MIN[co].head())

high          float64
volume        float64
asset          object
currency       object
delta         float64
delta_log     float64
return        float64
return_log    float64
interval       object
dtype: object
                       high     volume asset currency  delta  delta_log  \
date                                                                      
2021-01-03 18:58:00  728.83   16.31624   ETH      EUR    NaN        NaN   
2021-01-03 18:59:00  729.14   16.31751   ETH      EUR   0.31  -0.508638   
2021-01-03 19:00:00  728.79   92.63962   ETH      EUR  -0.35   0.455932   
2021-01-03 19:01:00  726.15  121.72397   ETH      EUR  -2.64  -0.421604   
2021-01-03 19:02:00  728.00   33.70983   ETH      EUR   1.85   0.267172   

                       return  return_log interval  
date                                                
2021-01-03 18:58:00       NaN         NaN   Minute  
2021-01-03 18:59:00  0.000425    0.000425   Minute  
2021-01-03 19:00:00 -0.000480   -0.000480   Minute

In [4]:
import pandas as pd

pd.read_csv("historical_data/BTC_EUR_Minute.csv")

,asset,curr,gran,high,rfc3339
0,BTC,EUR,Minute,26014.450000,2021-01-03T18:58:00Z
1,BTC,EUR,Minute,26087.320000,2021-01-03T18:59:00Z
2,BTC,EUR,Minute,26028.020000,2021-01-03T19:00:00Z
3,BTC,EUR,Minute,25981.860000,2021-01-03T19:01:00Z
4,BTC,EUR,Minute,26066.230000,2021-01-03T19:02:00Z
...,...,...,...,...,...
825047,BTC,EUR,Minute,98685.687500,2025-08-23T00:00:00Z
825048,BTC,EUR,Minute,98674.992188,2025-08-23T00:00:00Z
825049,BTC,EUR,Minute,98645.500000,2025-08-23T00:00:00Z
825050,BTC,EUR,Minute,98685.421875,2025-08-23T00:00:00Z


In [10]:
DAY.head()

,unix,volume,high,low,delta,delta_log,return,return_log,currency,asset,interval
date,,,,,,,,,,,
2014-09-17,1410912000,16389166.0,361.468506,351.586884,NaN,NaN,NaN,NaN,EUR,BTC,Day
2014-09-18,1410998400,26691849.0,355.505402,319.789459,-5.963104,-0.775472,-0.016497,-0.016497,EUR,BTC,Day
2014-09-19,1411084800,29560103.0,330.936707,298.921021,-24.568695,-1.390382,-0.069109,-0.069109,EUR,BTC,Day
2014-09-20,1411171200,28736826.0,329.978180,303.931244,-0.958527,0.018396,-0.002896,-0.002896,EUR,BTC,Day
2014-09-21,1411257600,20702625.0,321.504517,306.502197,-8.473663,-0.928071,-0.025679,-0.025679,EUR,BTC,Day


In [4]:
MIN.head()

,unix,volume,high,low,delta,delta_log,return,return_log,currency,asset,interval
date,,,,,,,,,,,
2021-01-03 18:58:00,1609700280,3.868866,26014.45,25952.41,NaN,NaN,NaN,NaN,EUR,BTC,Minute
2021-01-03 18:59:00,1609700340,11.543413,26087.32,25994.39,72.87,1.862549,0.002801,0.002801,EUR,BTC,Minute
2021-01-03 19:00:00,1609700400,2.461038,26028.02,25959.78,-59.30,-1.773055,-0.002273,-0.002273,EUR,BTC,Minute
2021-01-03 19:01:00,1609700460,1.388707,25981.86,25954.63,-46.16,-1.664266,-0.001773,-0.001773,EUR,BTC,Minute
2021-01-03 19:02:00,1609700520,6.743352,26066.23,25963.80,84.37,1.926188,0.003247,0.003247,EUR,BTC,Minute
